# Day 5 - Lab 2: Plan-and-Execute & Multi-Agent Systems

**Objective:** Explore advanced agent architectures, including the plan-and-execute model and conversational multi-agent systems using Microsoft's AutoGen.

**Estimated Time:** 135 minutes

**Introduction:**
In the previous lab, you built agents that could use tools. Now, we will explore how to orchestrate more complex agent behaviors. First, you will build a "plan-and-execute" agent that first thinks about a problem and then writes the code. Second, you will use the AutoGen framework to create a team of AI agents that can collaborate on a task through conversation.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will need to install the `pyautogen` library for the second part of this lab.

**Model Selection:**
For multi-agent systems and planning, models with strong reasoning and instruction-following are essential. `gpt-4.1` or `o3` are highly recommended for their planning capabilities.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to our individual planner/coder agents.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# This helper will install packages if they are not found
import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('pyautogen')

from utils import setup_llm_client, get_completion
import autogen

# AutoGen is optimized for the OpenAI API format.
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

## Step 2: The Challenges

### Challenge 1 (Foundational): A Plan-and-Execute Agent

**Task:** Create a two-step agent that first generates a detailed plan (a 'spec') for a Python function and then uses that plan to write the code.

> **Hint:** The 'Plan-and-Execute' pattern separates thinking from doing. The first agent (the Planner) creates a detailed blueprint. The second agent (the Coder) is given a much simpler task: just follow the blueprint. This separation often leads to more reliable and accurate results than asking a single agent to do everything at once.

**Instructions:**
1.  Define a high-level goal, such as: "Create a Python function that takes a list of strings and returns a new list containing only the strings that are palindromes."
2.  **Planner Agent:** Write a prompt that asks the LLM to act as a senior software architect. It should take the high-level goal and produce a detailed specification for the function, including the function signature, parameters, return value, and step-by-step logic.
3.  **Coder Agent:** Write a second prompt. This prompt should take the detailed specification from the Planner Agent as its *only* context and instruct the LLM to write the Python code that implements the spec.

**Expected Quality:** A two-stage generation process that separates the 'planning' from the 'doing', resulting in a well-defined Python function.

In [ ]:
high_level_goal = "Create a Python function that takes a list of strings and returns a new list containing only the strings that are palindromes."

# TODO: 1. Write the prompt for the Planner Agent.
# It should take the high_level_goal and produce a detailed function specification.
planner_prompt = f"""
# Your prompt here
"""

print("--- Planner Agent Generating Spec ---")
function_spec = get_completion(planner_prompt, client, model_name, api_provider)
print(function_spec)

# TODO: 2. Write the prompt for the Coder Agent.
# It should take the function_spec as context and write the final Python code.
coder_prompt = f"""
# Your prompt here
"""

print("\n--- Coder Agent Generating Code ---")
generated_function = get_completion(coder_prompt, client, model_name, api_provider)
print(generated_function)

### Challenge 2 (Intermediate): A Three-Agent AutoGen Team

**Task:** Use Microsoft's AutoGen framework to create a conversational team of three agents: a Product Manager, a Developer, and a User Proxy.

> **Tip:** The `UserProxyAgent` is special. It acts as your representative in the chat. Setting `human_input_mode` to `TERMINATE` tells the agent that it can stop the conversation on its own once it believes the task is complete, without asking you for confirmation.

**Instructions:**
1.  Define a `config_list` for AutoGen, which tells it which model to use.
2.  Create a `UserProxyAgent`. This agent represents you, the human user. Set its `human_input_mode` to `TERMINATE` so the conversation stops after a solution is proposed.
3.  Create an `AssistantAgent` named "ProductManager". Give it a system message defining its role (e.g., "You are a Product Manager. Your job is to clarify requirements and create a plan.").
4.  Create another `AssistantAgent` named "Developer". Give it a system message defining its role (e.g., "You are a senior Python developer. You write code based on the Product Manager's plan.").
5.  Create a `GroupChat` with all three agents and a `GroupChatManager`.
6.  Initiate the chat with a feature request from the user proxy, like "Add a feature to our API to calculate the complexity of a password."

**Expected Quality:** A conversational transcript showing the Product Manager clarifying the task, the Developer writing the code, and the process terminating successfully.

In [ ]:
# TODO: 1. Define the config_list for the LLM.
# Tip: This should be a list containing a dictionary with 'model' and 'api_key'.
config_list = [] # Your config list here

# TODO: 2. Create the UserProxyAgent.
# Tip: Use autogen.UserProxyAgent and set the human_input_mode.
user_proxy = None # Your agent here

# TODO: 3. Create the ProductManager agent.
# Tip: Use autogen.AssistantAgent and provide a system_message.
product_manager = None # Your agent here

# TODO: 4. Create the Developer agent.
developer = None # Your agent here

# TODO: 5. Create the GroupChat and GroupChatManager.
groupchat = None # Your group chat here
manager = None # Your manager here

# TODO: 6. Initiate the chat with a feature request.
feature_request = "Add a feature to our API to calculate the complexity of a password based on length, and the presence of uppercase, lowercase, numbers, and symbols."
# user_proxy.initiate_chat(...)

### Challenge 3 (Advanced): Multi-Agent System with a Code Reviewer

**Task:** Add a fourth agent, the `CodeReviewer`, to your AutoGen team. The conversation must now continue until the reviewer formally approves the developer's code.

**Instructions:**
1.  Keep the three agents from the previous challenge.
2.  Create a new `AssistantAgent` named "CodeReviewer". Its system message should instruct it to review Python code for quality, correctness, and adherence to best practices. It must end its review with the word "APPROVED" if the code is satisfactory.
3.  Modify the `UserProxyAgent`'s `is_termination_msg` property. This function should now check if the reviewer's last message contains the word "APPROVED".
4.  Create a new `GroupChat` with all four agents.
5.  Initiate the chat with the same feature request. Observe the conversation loop: the developer writes code, the reviewer critiques it, the developer revises the code, and the process repeats until the reviewer approves.

**Expected Quality:** A longer, more complex conversational transcript showing a collaborative loop of coding and reviewing, demonstrating a more realistic and robust development workflow.

In [ ]:
# TODO: 1. Create the CodeReviewer agent.
code_reviewer = None # Your agent here

# TODO: 2. Create a new UserProxyAgent with a custom termination message check.
# Tip: The `is_termination_msg` property accepts a lambda function.
user_proxy_with_review = autogen.UserProxyAgent(
    name="UserProxyWithReview",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=None # Your lambda function here
)

# TODO: 3. Create the new 4-agent GroupChat and Manager.
four_agent_groupchat = None # Your group chat here
four_agent_manager = None # Your manager here

# TODO: 4. Initiate the chat.
# user_proxy_with_review.initiate_chat(...)

## Lab Conclusion

Excellent work! You have now explored two powerful advanced agentic architectures. You learned how the plan-and-execute model can lead to more structured and reliable code generation, and you used AutoGen to simulate a collaborative team of AI agents that can plan, code, and review work. These foundational patterns are the building blocks for creating highly sophisticated and autonomous AI systems.

> **Key Takeaway:** Multi-agent systems allow you to break down a complex problem into smaller, more manageable tasks, each handled by a specialized AI agent. This division of labor, whether in a sequential 'plan-and-execute' pattern or a collaborative conversation, often leads to higher quality and more reliable results.